In [1]:
pip install spacy[transformers]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 366.9 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.5/53.5 kB 1.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 3.3 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [25]:
import json
preamble_data_file_ptr = open('/kaggle/input/lner-train-data/NER_TRAIN_PREAMBLE.json', 'r')
preamble_data = json.load(preamble_data_file_ptr)

In [27]:
judgement_data_file_ptr = open('/kaggle/input/lner-train-data/NER_TRAIN_JUDGEMENT.json', 'r')
judgement_data = json.load(judgement_data_file_ptr)

In [28]:
import random
data = preamble_data + judgement_data 
random.shuffle(data)

In [29]:
training_data = []
data_labels = ['annotations', 'result', 'text', 'data', 'entities', 'value', 'labels', 'start', 'end']

for text in data:
    
    annotation_len = len(text[data_labels[0]])
    results_len = len(text[data_labels[0]][0][data_labels[1]])
    
    if annotation_len > 0 and results_len > 0:
        
        info_dict = {}
        info_dict[data_labels[2]] = text[data_labels[3]][data_labels[2]]
        info_dict[data_labels[4]] = []
        
        for data in text[data_labels[0]][0][data_labels[1]]:
            
            label = data[data_labels[5]][data_labels[6]][0]
            label_start = data[data_labels[5]][data_labels[7]]
            label_end = data[data_labels[5]][data_labels[8]]
            info_dict[data_labels[4]].append([label_start, label_end, label])
        
        training_data.append(info_dict)
        
print(training_data[0])

{'text': 'There can be no doubt that the object of making a special provision for the advancement of the castes or communities, there specified, is to carry out the directive principle enshrined in Art. 46.', 'entities': [[188, 195, 'PROVISION']]}


In [30]:
import spacy
from tqdm import tqdm
from spacy.tokens import DocBin

#Load a new spacy model
nlp = spacy.blank("en") 
#Create a DocBin object
doc_bin = DocBin()

In [31]:
from spacy.util import filter_spans

for training_text in tqdm(training_data): 
    
    text = training_text[data_labels[2]]
    labels = training_text[data_labels[4]]
    doc = nlp.make_doc(text)
    entities = []
    
    for start, end, label in labels:
        span = doc.char_span(start, end, label = label, alignment_mode = "contract")
        if span is None:
            print("Skipping the entity")
        else:
            entities.append(span)
    
    filtered_entities = filter_spans(entities)
    doc.ents = filtered_entities
    doc_bin.add(doc)


#Save the DocBin object
doc_bin.to_disk("training_data.spacy") 

100%|██████████| 8816/8816 [00:06<00:00, 1460.12it/s]


In [33]:
!python -m spacy init fill-config /kaggle/input/lner-config-file/base_config.cfg config.cfg

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [34]:
!python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy --gpu-id 0

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
ℹ Saving to output directory: .
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2022-12-10 08:12:33,806] [INFO] Set up nlp object from config
[2022-12-10 08:12:33,816] [INFO] Pipeline: ['transformer', 'ner']
[2022-12-10 08:12:33,821] [INFO] Created vocabulary
[2022-12-10 08:12:33,822] [INFO] Finished initializing nlp object
Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.weight']
- This IS expected if you are initializ

In [35]:
import json
preamble_dev_data_file_ptr = open('/kaggle/input/lner-dev-data/NER_DEV_PREAMBLE.json', 'r')
preamble_dev_data = json.load(preamble_dev_data_file_ptr)

In [36]:
judgement_dev_data_file_ptr = open('/kaggle/input/lner-dev-data/NER_DEV_JUDGEMENT.json', 'r')
judgement_dev_data = json.load(judgement_dev_data_file_ptr)

In [40]:
dev_data = preamble_dev_data + judgement_dev_data
random.shuffle(dev_data)

In [47]:
nlp_ner = spacy.load("model-best")

doc = nlp_ner(dev_data[2][data_labels[3]][data_labels[2]])
colors = {"COURT": "#F67DE3", "JUDGE": "#7DFFD9", "PETITIONER": "#FFFFFF", "RESPONDENT": "#EDF6E2", "LAWYER": "#FBE5D9", "DATE": "#7FB6A9", "ORG": "#7DF6EA", "GPE": "#9DF6D9", "STATUTE": "#7FF8D9", "PROVISION": "#7DF6D2", "PRECEDENT": "#23F2D9", "CASE_NUMBER": "#7DF5D8", "WITNESS": "#7EE5D8", "OTHER_PERSON": "#23F5D8"}
options = {"colors": colors} 

spacy.displacy.render(doc, style = "ent", options = options, jupyter = True)